<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Подбор-гиперпараметров-на-выборке-с-устраненным-дисбалансом" data-toc-modified-id="Подбор-гиперпараметров-на-выборке-с-устраненным-дисбалансом-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Подбор гиперпараметров на выборке с устраненным дисбалансом</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

ПРИЗНАКИ:

RowNumber — индекс строки в данных

CustomerId — уникальный идентификатор клиента

Surname — фамилия

CreditScore — кредитный рейтинг

Geography — страна проживания

Gender — пол

Age — возраст

Tenure — сколько лет человек является клиентом банка

Balance — баланс на счёте

NumOfProducts — количество продуктов банка, используемых клиентом

HasCrCard — наличие кредитной карты

IsActiveMember — активность клиента

EstimatedSalary — предполагаемая зарплата

ЦЕЛЕВОЙ ПРИЗНАК:

Exited — факт ухода клиента

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Привожу к регистру

In [4]:
data.columns = ['row_number','customer_id','surname','credit_score','geography','gender','age','tenure','balance','num_of_products','has_cr_card','is_active_member','estimated_salary','exited']

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
data.describe()

,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Наблюдаем пропуски с строке Tenure — сколько лет человек является клиентом банка, скорее всего это связанно с некорректным сбором информации. 

In [7]:
print('Доля пропусков к общему количеству:', data['tenure'].isna().mean())

Доля пропусков к общему количеству: 0.0909


Заменю медианой

In [8]:
data.loc[data.tenure.isna(),'tenure']=data.loc[data.tenure.isna()==False,'tenure'].median()

Некоторые данные не важны для обучения и решения задачи - удаляю

In [9]:
data_learn = data.drop(['row_number','customer_id','surname'],axis=1)
data_learn.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [10]:
data_learn = pd.get_dummies(data_learn,drop_first=True)

In [11]:
data_learn.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Вывод:данный проанализированы - недостающие заменены медианным значениям. 
Данные не влияющие на результат выполнения - удалены.     

## Исследование задачи

Сформировал выборку

In [12]:
features = data_learn.drop('exited',axis=1)
target=data_learn['exited']

In [13]:
features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.2,random_state=12345)

In [14]:
scaler = StandardScaler()

numeric = ['credit_score', 'age', 'balance', 'num_of_products', 'estimated_salary', 'tenure']

scaler.fit(features_train[numeric])

features_train.loc[:,(numeric)] = scaler.transform(features_train.loc[:,(numeric)])
features_test.loc[:,(numeric)] = scaler.transform(features_test.loc[:,(numeric)])

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [15]:
print('Значения тренировочной выборке')
print(target_train.value_counts(normalize=True))
print('Значений тестовой выборке')
print(target_test.value_counts(normalize=True))

Значения тренировочной выборке
0    0.79875
1    0.20125
Name: exited, dtype: float64
Значений тестовой выборке
0    0.7865
1    0.2135
Name: exited, dtype: float64


In [16]:
model_regre = LogisticRegression(random_state=12345,solver='liblinear')
model_regre.fit(features_train, target_train)
predictions_test = model_regre.predict(features_test)

probabilities_test = model_regre.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('F1=',f1_score(target_test,predictions_test))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_test))

F1= 0.2857142857142857
ROC-AUC= 0.7575583879607725


In [17]:
model_tree = DecisionTreeClassifier(random_state=12345)
model_tree.fit(features_train,target_train)
predictions_test = model_tree.predict(features_test)

probabilities_test = model_tree.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('F1=',f1_score(target_test,predictions_test))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_test))

F1= 0.5091743119266056
ROC-AUC= 0.6890694997997533


Данные модели нам не подходят

<b>Вывод по логистической регрессии</b></p>
Использование Логистической регрессии показывает невысокие значения метрик в силу отсутствия заметной линейной связи между свойствами и целевым признаком.

In [18]:
model_forest = RandomForestClassifier(random_state=12345)
model_forest.fit(features_train,target_train)
predictions_test = model_forest.predict(features_test)

probabilities_test = model_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('F1=',f1_score(target_test,predictions_test))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_test))
print('Accuracy on train_data',model_forest.score(features_train,target_train))

F1= 0.5672082717872968
ROC-AUC= 0.8550585033446434
Accuracy on train_data 1.0


<b>Вывод по выбору модели</b> Очевидно, что наиболее подходящей моделью является "случайный лес". Однако значения accuracy на учебной выборке показывают переобучение. При этом применение корректировки дисбаланса для линейной регрессии привело к существенному улучшению показателей, для дерева решений/случайного леса к незначительному (на уровне погрешности) ухудшению.
Целесообразно осуществить настройку гиперпараметров для исключения переобучения и возможно улучшения показателей F1 и ROC-AUC, при одновременном устранении дисбаланса, что также может повысить показатели модели. 

<b>RandomizedSearchCV</b>

In [19]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 400, num = 10)]
max_features = ['log2', 'sqrt'] 
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]
bootstrap = [True, False]

In [20]:
param_dist = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

In [21]:
model_forest_tune = RandomForestClassifier(random_state=12345)
rs = RandomizedSearchCV(model_forest_tune, param_dist, n_iter = 300,  
                        scoring='f1', 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=1, random_state=12345)

In [22]:
rs.fit(features_train, target_train)
rs.best_params_

Fitting 3 folds for each of 300 candidates, totalling 900 fits


{'n_estimators': 266,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 15,
 'bootstrap': True}

In [23]:
model_forest2 = RandomForestClassifier(random_state=12345,n_estimators = 266,
                                       min_samples_split = 6,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=15
                                      ,bootstrap=False)
model_forest2.fit(features_train, target_train)
predictions_2 = model_forest2.predict(features_test)
probabilities_2 = model_forest2.predict_proba(features_test)
probabilities_one_2 = probabilities_2[:, 1]

print('F1=',f1_score(target_test,predictions_2))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_2))
print('Accuracy on train_data',model_forest2.score(features_train,target_train))
print('Accuracy on test_data',model_forest2.score(features_test,target_test))

F1= 0.556547619047619
ROC-AUC= 0.8586212595154472
Accuracy on train_data 0.967375
Accuracy on test_data 0.851


снизим глубину, чтобы не допустить переобучения 

In [24]:
model_forest2 = RandomForestClassifier(random_state=12345,n_estimators = 366,
                                       min_samples_split = 4,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=7
                                      ,bootstrap=False)
model_forest2.fit(features_train, target_train)
predictions_2 = model_forest2.predict(features_test)
probabilities_2 = model_forest2.predict_proba(features_test)
probabilities_one_2 = probabilities_2[:, 1]

print('F1=',f1_score(target_test,predictions_test))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_2))
print('Accuracy on train_data',model_forest2.score(features_train,target_train))
print('Accuracy on test_data',model_forest2.score(features_test,target_test))

F1= 0.5672082717872968
ROC-AUC= 0.866973562949718
Accuracy on train_data 0.87625
Accuracy on test_data 0.851


In [25]:
rs.best_score_

0.5731319353521812

In [26]:
search_data = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
search_data = search_data.drop([
            'mean_fit_time', 
            'std_fit_time',
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
search_data.head(10)

,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_bootstrap,mean_test_score,rank_test_score
0,266,6,2,sqrt,15,True,0.573132,1
1,266,8,2,sqrt,11,False,0.572226,2
2,366,4,2,sqrt,15,False,0.572152,3
3,100,10,2,log2,14,False,0.572007,4
4,233,12,2,sqrt,11,True,0.571631,5
5,233,10,2,log2,11,False,0.571168,6
6,200,6,4,sqrt,13,False,0.571153,7
7,400,6,2,sqrt,14,True,0.570856,8
8,166,6,8,log2,11,False,0.570148,9
9,300,18,2,log2,12,False,0.570133,10


GridSearchCV 

In [27]:
n_estimators = [133,266,366]
max_features = ['sqrt']
max_depth = [6,7,8]
min_samples_split = [6,8,14]
min_samples_leaf = [2,8,16]
bootstrap = [True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(model_forest_tune, param_grid, cv = 3, verbose = 1, n_jobs=1, scoring='f1')
gs.fit(features_train, target_train)
model_forest_cv = gs.best_estimator_
gs.best_params_

Fitting 3 folds for each of 81 candidates, totalling 243 fits


{'bootstrap': True,
 'max_depth': 8,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 14,
 'n_estimators': 366}

Проверим метрики подобранные GridSearchCV

In [ ]:
model_forest3 = RandomForestClassifier(random_state=12345,n_estimators = 366,
                                       min_samples_split = 14,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=8
                                      ,bootstrap=False)
model_forest3.fit(features_train, target_train)
predictions_3 = model_forest3.predict(features_test)
probabilities_3 = model_forest3.predict_proba(features_test)
probabilities_one_3 = probabilities_3[:, 1]

print('F1=',f1_score(target_test,predictions_3))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_3))
print('Accuracy on train_data',model_forest3.score(features_train,target_train))


Подбор гиперпараметров не помог.Требуется устранение дисбалланса.

## Борьба с дисбалансом

Весовые коэффициенты

In [29]:
model_regre = LogisticRegression(random_state=12345,solver='liblinear',class_weight='balanced')
model_regre.fit(features_train, target_train)
predictions = model_regre.predict(features_test)

probabilities = model_regre.predict_proba(features_test)
probabilities_one = probabilities[:, 1]
print('F1=',f1_score(target_test,predictions))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one))

F1= 0.5020990764063813
ROC-AUC= 0.7635747263169022


In [30]:
model_tree = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model_tree.fit(features_train,target_train)
predictions = model_tree.predict(features_test)

probabilities = model_tree.predict_proba(features_test)
probabilities_one = probabilities[:, 1]
print('F1=',f1_score(target_test,predictions))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one))
print('Accuracy',model_tree.score(features_train,target_train))

F1= 0.5029940119760478
ROC-AUC= 0.6829645764071994
Accuracy 1.0


In [31]:
model_forest = RandomForestClassifier(random_state=12345,n_estimators = 366,
                                       min_samples_split = 14,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=8
                                      ,bootstrap=False, class_weight='balanced')
model_forest.fit(features_train,target_train)
predictions = model_forest.predict(features_test)

probabilities = model_forest.predict_proba(features_test)
probabilities_one = probabilities[:, 1]

print('F1=',f1_score(target_test,predictions))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one))
print('Accuracy on train_data (overeducation)',model_forest.score(features_train,target_train))

F1= 0.6289682539682541
ROC-AUC= 0.8686589118779879
Accuracy on train_data (overeducation) 0.8415


Upsampling

In [32]:
def upsample(features, targets, ratio):
    features_zero = features[targets==0]
    features_ones = features[targets==1]
    target_zero = targets[targets==0]
    target_ones = targets[targets==1]
    features_multiple = pd.concat([features_zero]+[features_ones]*ratio)
    target_multiple = pd.concat([target_zero]+[target_ones]*ratio)
    #перемешиваем полученный массив значений
    features_multiple, target_multiple = shuffle(
        features_multiple, target_multiple, random_state=12345)
    return features_multiple, target_multiple

In [33]:
ratio_upsample = 4
features_train_up, target_train_up = upsample(features_train,target_train, ratio_upsample)
print('Доля объектов - клиентов прекративших обслуживание в сформированной учебной выборке',target_train_up.mean())
display(features_train_up.head())
target_train_up

Доля объектов - клиентов прекративших обслуживание в сформированной учебной выборке 0.5019485580670304


,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
3683,-0.772591,1.247608,1.447547,1.505952,0.811234,0,0,0.836331,0,0,0
21,-0.152753,-0.654125,1.084549,-1.220083,0.811234,1,0,0.664211,0,1,0
1111,1.882383,1.913215,-1.456441,0.875418,-0.899779,1,1,-1.206378,1,0,1
6314,1.996020,0.201655,1.810546,0.002098,-0.899779,1,1,0.709655,0,0,0
404,0.353449,-1.509905,1.084549,-1.220083,0.811234,1,1,0.207697,0,0,1


3683    1
21      0
1111    1
6314    1
404     0
       ..
5686    1
4526    0
229     0
3744    1
4682    0
Name: exited, Length: 12830, dtype: int64

### Подбор гиперпараметров на выборке с устраненным дисбалансом

In [34]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 400, num = 10)]
max_features = ['log2', 'sqrt'] 
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]
bootstrap = [True, False]

In [35]:
param_dist = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

In [36]:
model_forest_tune = RandomForestClassifier(random_state=12345)
rs = RandomizedSearchCV(model_forest_tune, param_dist, n_iter = 300,  
                        scoring='f1', 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=1, random_state=12345)

In [ ]:
rs.fit(features_train_up, target_train_up)
rs.best_params_

Fitting 3 folds for each of 300 candidates, totalling 900 fits


In [ ]:
model_forest2 = RandomForestClassifier(random_state=12345,n_estimators = 366,
                                       min_samples_split = 4,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=15
                                      ,bootstrap=False)
model_forest2.fit(features_train_up, target_train_up)
predictions_2 = model_forest2.predict(features_test)
probabilities_2 = model_forest2.predict_proba(features_test)
probabilities_one_2 = probabilities_2[:, 1]

print('F1=',f1_score(target_test,predictions_2))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_2))
print('Accuracy on train_data (overeducation)',model_forest2.score(features_train_up,target_train_up))
print('Accuracy on test_data (overeducation)',model_forest2.score(features_test,target_test))

На основных рекомендованных по результатам RandomSearch мы имеем достаточное значение f1-меры (0.6086>0.59) но высокое значение accuracy на учебной выборке (0,995) свидетельствуещеее о переобучени. 

In [ ]:
model_forest2 = RandomForestClassifier(random_state=12345,n_estimators = 366,
                                       min_samples_split = 4,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=10
                                      ,bootstrap=False)
model_forest2.fit(features_train_up, target_train_up)
predictions_2 = model_forest2.predict(features_test)
probabilities_2 = model_forest2.predict_proba(features_test)
probabilities_one_2 = probabilities_2[:, 1]

print('F1=',f1_score(target_test,predictions_test))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_2))
print('Accuracy on train_data',model_forest2.score(features_train_up,target_train_up))
print('Accuracy on test_data ',model_forest2.score(features_test,target_test))

Предварительная проверка показывает положительные изменения метрик по результатам подбора гиперпараметров

In [ ]:
search_data = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
search_data = search_data.drop([
            'mean_fit_time', 
            'std_fit_time',
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
search_data.head(10)

<b>GridSearchCV</b>

In [ ]:
n_estimators = [100,133,266,366]
max_features = ['sqrt']
max_depth = [6,7,8]
min_samples_split = [6,8,12]
min_samples_leaf = [2,6,8]
bootstrap = [True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(model_forest_tune, param_grid, cv = 3, verbose = 1, n_jobs=1, scoring='f1')
gs.fit(features_train_up, target_train_up)
model_forest_cv = gs.best_estimator_
gs.best_params_

In [ ]:
По результатам отработки GridSearchCV получены значения на основании которых следует провести тестирование модели.

## Тестирование модели

In [ ]:
model_forest3 = RandomForestClassifier(random_state=12345,n_estimators = 266,
                                       min_samples_split = 6,
                                       min_samples_leaf = 2,
                                      max_features='sqrt', max_depth=8
                                      ,bootstrap=False)
model_forest3.fit(features_train_up, target_train_up)
predictions_3 = model_forest3.predict(features_test)
probabilities_3 = model_forest3.predict_proba(features_test)
probabilities_one_3 = probabilities_3[:, 1]

print('F1=',f1_score(target_test,predictions_3))
print('ROC-AUC=',roc_auc_score(target_test,probabilities_one_3))
print('Accuracy on train_data (overeducation)',model_forest3.score(features_train_up,target_train_up))
print('Accuracy on valid_data (overeducation)',model_forest3.score(features_test,target_test))

Вывод
Проанализированы представленные заказчиком данные о 10000 клиентов Банка:

Некорректные данные не установлены
Установлены пропуски по параметру "срок обслуживания" - tenure. Данные пропуски заполнены медианным значениям
Проведено категорирование соответствующих признаков (Geography, Gender)
Проверена работа моделей на датасете. Проведен подбор гиперпараметров. Целевые значения метрик не достигнуты.
Установлен дисбалланс классов и проведена работа по его устранению.
По результатам устранения дисбалланса, переобученная модель RandomForestClassifier показала на тестовой выборке значение f1 = 0.629 что превышает целевое значение в 0,59.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*